In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.utils import image_dataset_from_directory
from keras import layers
import matplotlib.pyplot as plt
from tensorflow.keras.metrics import Precision, Recall
import zipfile
import os
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [ ]:
with zipfile.ZipFile("/content/Dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("Dataset")

In [ ]:
test_data = image_dataset_from_directory("/content/Dataset/Dataset/Test",image_size = (224,224),batch_size = 128*2)


Found 80362 files belonging to 2 classes.


In [ ]:
train_data = image_dataset_from_directory("/content/Dataset/Dataset/Train",image_size = (224,224),batch_size = 50)


Found 120542 files belonging to 2 classes.


In [ ]:
inputs = keras.Input(shape= (224,224,3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters = 32,kernel_size = 3,activation = "relu")(x)
x= layers.MaxPooling2D(pool_size = 2)(x)
x = layers.Conv2D(filters = 64,kernel_size = 3,activation = "relu")(x)
x= layers.MaxPooling2D(pool_size = 2)(x)
x = layers.Conv2D(filters = 128,kernel_size = 3,activation = "relu")(x)
x= layers.MaxPooling2D(pool_size = 2)(x)
x = layers.Conv2D(filters = 256,kernel_size = 3,activation = "relu")(x)
x= layers.MaxPooling2D(pool_size = 2)(x)
x = layers.Conv2D(filters = 512,kernel_size = 3,activation = "relu")(x)
x= layers.MaxPooling2D(pool_size = 2)(x)
x = layers.Flatten()(x)
x = layers.Dense(128,activation = "relu")(x)
x = layers.Dense(56,activation = "relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1,activation ="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs = outputs)

In [ ]:
earlystop_cb = EarlyStopping(
    monitor='loss',
    patience=3,              # Wait for 5 epochs without improvement
    restore_best_weights=True,  # Load best weights after stopping
    verbose=1
)

In [ ]:
model.compile(optimizer="adam" ,loss ="binary_crossentropy",metrics =["accuracy", Precision(name='precision'), Recall(name='recall')])

In [ ]:
model.fit(train_data,epochs = 50,callbacks = [earlystop_cb])


Epoch 1/50
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 45s 17ms/step - accuracy: 0.8611 - loss: 0.3570 - precision: 0.7338 - recall: 0.5115
Epoch 2/50
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 38s 16ms/step - accuracy: 0.8728 - loss: 0.3313 - precision: 0.7651 - recall: 0.5574
Epoch 3/50
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 38s 16ms/step - accuracy: 0.8742 - loss: 0.3285 - precision: 0.7696 - recall: 0.5599
Epoch 4/50
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 38s 16ms/step - accuracy: 0.8747 - loss: 0.3267 - precision: 0.7732 - recall: 0.5588
Epoch 5/50
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 38s 16ms/step - accuracy: 0.8741 - loss: 0.3255 - precision: 0.7717 - recall: 0.5570
Epoch 6/50
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 38s 16ms/step - accuracy: 0.8745 - loss: 0.3250 - precision: 0.7715 - recall: 0.5591
Epoch 7/50
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 38s 16ms/step - accuracy: 0.8758 - loss: 0.3230 - precision: 0.7756 - recall: 0.5624
Epoch 8/50
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 38s 16ms/step - accuracy: 0.8759 - loss: 0.3228 - precision: 0.7793 -

In [ ]:
all_labels = []
for i,l in test_data:
  all_labels.extend(l.numpy())

In [ ]:
clean_labels = [int(x) for x in all_labels]

In [ ]:
y_pred = model.predict(test_data)

314/314 ━━━━━━━━━━━━━━━━━━━━ 16s 37ms/step


In [ ]:
y_pred_labels = (y_pred > 0.5).astype(int).flatten()


In [ ]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,accuracy_score

In [ ]:
accuracy_score(clean_labels,y_pred_labels)

0.7081207535900053

In [ ]:
recall_score(clean_labels,y_pred_labels)

0.1423444976076555

In [ ]:
precision_score(clean_labels,y_pred_labels)

0.2070285316631872

In [ ]:
f1_score(clean_labels,y_pred_labels)

0.1686986107173235

In [ ]:
confusion_matrix(clean_labels,y_pred_labels)

array([[54526,  9116],
       [14340,  2380]])